# TECNICA PRO: PIPELINES

## OBJETIVO

 3 principales conceptos y clases que vamos a necesitar para organizar nuestro proyecto en un proceso fácilmente replicable y ejecutable:

1. make_pipeline
2. make_column_transformer
3. FunctionTransformer

## SET UP

### Importación y configuración

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pickle

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer

from sklearn.linear_model import LinearRegression

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer

#Automcompletar rápido
%config IPCompleter.greedy=True

### Carga de datos

In [ ]:
df = pd.read_csv('../../00_DATASETS/sintetico_continua.csv')
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,target
0,-0.333643,0.385530,1.733067,0.457594,0.047734,0.057823,0.986681,-0.310969,2.492347,137.101716
1,-0.915244,0.182435,0.962427,0.285146,0.538592,0.604320,1.274675,-1.306736,1.274135,43.190649
2,0.719608,-0.840527,-1.999640,-0.893353,0.826154,-1.076738,-1.902825,0.094109,0.414780,-78.126159
3,-1.040670,-0.318717,-1.332247,-0.605161,-0.724380,-0.103789,0.767642,1.352745,1.483947,-219.544675
4,0.118715,-0.593328,1.836431,-0.183321,-0.558974,-0.989050,1.208619,0.762238,0.775032,139.807003
...,...,...,...,...,...,...,...,...,...,...
995,-1.466331,0.138283,-1.069003,-0.729966,0.248520,1.901812,-0.503945,1.018632,-0.580477,-140.829803
996,1.503471,1.702940,-1.562491,-1.218632,1.402215,1.865504,-1.020819,0.441283,-1.775826,65.138366
997,0.148292,-0.829697,-0.192043,-0.204354,0.544293,1.538875,-0.119157,-1.241458,1.042977,-14.257548
998,0.571149,0.955130,0.871865,0.689523,-0.163711,1.984590,-1.180553,0.297048,-0.118282,137.454992


### Creación de X e y

In [ ]:
x = df.drop(columns='target').copy()
y = df['target']

### Separación train y test

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.3)

## PIPELINES

### ¿Qué es un pipeline?

Es una funcionalidad de ScikitLearn que permite encadenar procesos en serie.

La salida del proceso actual será la entrada del proceso siguiente.

Si guardamos un pipeline guardará todo el proceso que contiene, incluyendo tanto la parte de preprocesamiento como la de modelización.

Y si lo recuperamos y ejecutamos ejecutará todo el proceso.

Eso hace más sencillo el reentremiento y la ejecución de todo el flujo del proyecto de machine learning.



### ¿Qué tipos de pipeline existen?

Un pipeline puede tener todos los pasos que queramos, tanto tranformers como predictors, pero sólo puede haber un único predictor que, de existir, irá al final.

Esto hace que haya en la práctica dos tipos de pipe:

* los que solo tienen transformers: desde uno hasta todos los queramos
* los que tienen un predictor

Si solo tienen transformers podremos usar los métodos: fit(), transform(), fit_transform()

Si tienen un predictor podremos usar los métodos: fit(), predict(), fit_predict()

### ¿Cómo se crea y usa un pipeline?

#### Instanciar todos los procesos que se vayan a utilizar

In [ ]:
#Instanciamos un discretizer
discretizer = KBinsDiscretizer(n_bins = 10, strategy = 'uniform', encode = 'ordinal')

#Instanciamos un modelo
rm = LinearRegression()

#### Instanciar el pipeline

Hay que poner los procesos en el orden deseado y separados por comas.

Usaremos la clase make_pipeline:

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html?highlight=make_pipe#sklearn.pipeline.make_pipeline

In [ ]:
pipe = make_pipeline(discretizer,rm)

A partir de aquí el pipeline funciona como un modelo.

#### Entrenar el pipeline

In [ ]:
pipe.fit(train_x,train_y)

Pipeline(steps=[('kbinsdiscretizer',
                 KBinsDiscretizer(encode='ordinal', n_bins=10,
                                  strategy='uniform')),
                ('linearregression', LinearRegression())])

#### Predecir con el pipeline

In [ ]:
pipe.predict(test_x)[:5]

array([ 311.78219937,   -7.69213669, -298.2275947 ,  188.11754803,
       -337.21729208])

#### Guardar un pipeline

Podemos guardar un pipeline para usarlo posteriormente igual que un modelo.

Es decir con pickle o con joblib.

In [ ]:
with open('mi_pipeline.pickle', mode='wb') as archivo:
   pickle.dump(pipe, archivo)

#### Cargar un pipeline

NOTA: Para cargar un pipeline guardado y que funcione el entorno debe ser igual al entorno en el que se creó (igual que con los modelos).

Al predecir con un pipeline ejecutará tanto el modelo como las transformaciones anteriores.

In [ ]:
with open('mi_pipeline.pickle', mode='rb') as archivo:
   pipe2 = pickle.load(archivo)

In [ ]:
pipe2.predict(test_x)[:5]

array([ 311.78219937,   -7.69213669, -298.2275947 ,  188.11754803,
       -337.21729208])

### ¿Cual es la diferencia entre make_pipeline y Pipeline?

Con make_pipeline él pone internamente un nombre a cada paso (que será la versión en minúscula del nombre de la clase).

Mientras que con Pipeline tú tienes que ponerle un nombre a cada paso.

En la sintaxis de Pipiline tienes que pasarle todos los pasos del proceso en una lista, y cada uno de esos pasos tiene que ser una tupla que contenga como primer elemento el nombre que le quieras poner y como segundo elemento la instancia del objeto.

Esto hace que la sintaxis de make_pipeline sea bastante más sencilla, lo cual en proyectos complejos es una ventaja.

Sintaxis de make_pipeline:
pipe = make_pipeline(discretizer,rm)

Sintaxis de Pipeline:
pipe = Pipeline([('discretizador',discretizer),('regresion',rm)])

Exactamente lo mismo aplica a make_column_transformer.

### ¿Qué es y cómo se usa make_column_transformer?

Un pipeline hace operaciones en serie, pero ¿qué pasa si queremos hacer operaciones en paralelo?

Por ejemplo, hemos visto que muchas veces tenemos que aplicar el diferentes procesos a diferentes columnas.

Como cuando aplicamos ohe a unas variables, oe a otras, etc.

Sin embargo en un pipe se aplica cada paso del proceso a todo el dataframe que le pasemos como input.

Para solucinarlo usamos make_column_transformer, que permite especificar a qué columnas concretas queremos que se aplique cada proceso.

https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_transformer.html

Su sintaxis es:

* Usar una tupla para cada proceso
* Cada tupla contiene dos elementos separados por comas:
    * la instancia del proceso
    * las variables sobre las que se aplica
* Si no se cubren todas las variables hay que decirle qué hacer con el resto mediante el parámetro remainder:
    * si lo ponemos a 'drop' (defecto) las eliminará
    * si lo ponemos a 'passthrough' las pasará al siguiente step tal cual están

Por ejemplo, en lugar de aplicar el discretizer a todas las variables vamos a aplicar:

* KBinsDiscretizer de x1 a x5
* Binarizer a x6 y x7
* No hacer nada sobre x8 y x9

Usaremos las instancias que ya teníamos creadas, y creamos la del binarizer.

In [ ]:
bin = Binarizer(threshold=0)

Ahora instanciamos un column transformer que identifique qué proceso se aplica sobre qué variables.

In [ ]:
ct = make_column_transformer(
            (discretizer,['x1','x2','x3','x4','x5']),
            (bin,['x6','x7']),
            remainder = 'passthrough')

Por último incluímos el column transformer como un paso más dentro del pipe.

Pero ahora se aplicará cada proceso a las variables que hemos definido.

In [ ]:
pipe = make_pipeline(ct,rm)

pipe.fit(train_x,train_y)

pipe.predict(test_x)[:5]

array([ 300.29114397,  -17.69143885, -284.66554416,  197.40511134,
       -326.90396094])

### ¿Qué es y cómo se usa FunctionTransformer?

De forma nativa podemos incluir dentro de pipelines y column transformers todos los pasos que se ejecuten con algún transformer o predictor de scikit learn.

Pero no funciona fuera de sklearn, es decir, no vale para funciones de numpy o pandas, ni las existentes ni las personalizadas.

Entonces para poder incluir las partes de nuestro proyecto que no son sklearn dentro del pipe tenemos que convertir esas funciones a transformers.

Lo hacemos con FunctionTransformer:

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html

Esta clase permite crear un transformer que funciona en sklearn a partir de una función que ya exista (como dropna) o que sea una nuestra personalizada.

Como ejemplo vamos a crear una función que modifique una variable restándole una unidad y la vamos a incluir en el pipeline.

In [ ]:
# Definimos la función personalizada
def restar_uno(variable):
    variable = variable - 1
    return(variable)

# La pasamos a un transformer
restar_uno_ft = FunctionTransformer(restar_uno)

# La inclimos dentro del pipe como si fuera ya un transformer de sklearn
ct = make_column_transformer(
            (restar_uno_ft,['x2']),
            (discretizer,['x1','x2','x3','x4','x5']),
            (bin,['x6','x7']),
            remainder = 'passthrough')

pipe = make_pipeline(ct,rm)

pipe.fit(train_x,train_y)

pipe.predict(test_x)[:5]

array([-110.76122137,  148.19451529,   93.67859688,  -96.63317074,
        -10.8270417 ])

## OPERACIONES AVANZADAS

### Operaciones avanzadas con un pipeline

#### Cómo ver lo que pasa dentro del pipeline

En general no lo necesitaremos.

Pero podemos acceder a cada componente del pipeline usando .named_steps

In [ ]:
pipe.named_steps

{'columntransformer': ColumnTransformer(remainder='passthrough',
                   transformers=[('kbinsdiscretizer',
                                  KBinsDiscretizer(encode='ordinal', n_bins=10,
                                                   strategy='uniform'),
                                  ['x1', 'x2', 'x3', 'x4', 'x5']),
                                 ('binarizer', Binarizer(threshold=0),
                                  ['x6', 'x7'])]),
 'linearregression': LinearRegression()}

Podemos seguir navegando por los componentes usando los nombres que nos devuelve.

Por ejemplo si quisiéramos extraer los coeficientes de la regresión:

In [ ]:
pipe.named_steps.linearregression.coef_

array([-7.1181884 , 39.06017886,  3.39062079, 50.03159976, 13.4189827 ,
       52.06766481, -2.69076004,  0.37049021, 34.43113606,  0.64742246])

O podemos navegar el columntransformer que es más complejo, por ejemplo viendo las transformaciones que incluye y a qué variables se las aplica.

In [ ]:
pipe.named_steps.columntransformer.get_params

<bound method ColumnTransformer.get_params of ColumnTransformer(remainder='passthrough',
                  transformers=[('kbinsdiscretizer',
                                 KBinsDiscretizer(encode='ordinal', n_bins=10,
                                                  strategy='uniform'),
                                 ['x1', 'x2', 'x3', 'x4', 'x5']),
                                ('binarizer', Binarizer(threshold=0),
                                 ['x6', 'x7'])])>

#### Representación visual de un pipeline

In [ ]:
from sklearn import set_config
set_config(display = 'diagram')

In [ ]:
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('kbinsdiscretizer',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   n_bins=10,
                                                                   strategy='uniform'),
                                                  ['x1', 'x2', 'x3', 'x4',
                                                   'x5']),
                                                 ('binarizer',
                                                  Binarizer(threshold=0),
                                                  ['x6', 'x7'])])),
                ('linearregression', LinearRegression())])

#### Guardar y cargar un pipeline que incuya funciones personalizadas

Si guardamos un pipeline que incluya alguna función personalizada (que habremos metido con FunctionTransformer) tenemos que volver a crear esa función en el notebook en el que usemos el pipeline.

Pero hay una alternativa a pickle que guarda automáticamente también las funciones personalizadas, por lo que ahorra mucho trabajo.

Se llama cloudpickle, y funciona exactamente igual que pickle, solo tenemos que cambiar el nombre.

In [ ]:
import cloudpickle

In [ ]:
#Guardar un pipeline con cloudpickle
with open('mi_pipeline.pickle', mode='wb') as archivo:
   cloudpickle.dump(pipe, archivo)

In [ ]:
#Cargar un pipeline con cloudpickle
with open('mi_pipeline.pickle', mode='rb') as archivo:
   pipe3 = cloudpickle.load(archivo)

pipe3.predict(test_x)[:5]

array([ 300.29114397,  -17.69143885, -284.66554416,  197.40511134,
       -326.90396094])

### Operaciones avanzadas con un column transformer

#### Diferentes formas de seleccionar variables en un column transformer

Cuando tenemos muchas variables no es eficiente tener que escribir todas las que queremos incluir en un paso del column transformer.

Por ello conviene conocer formas más avanzadas que pueden ser útiles.

In [ ]:
set_config(display = 'text')

In [ ]:
#Por posición
make_column_transformer((discretizer,[0,1,2]))

ColumnTransformer(transformers=[('kbinsdiscretizer',
                                 KBinsDiscretizer(encode='ordinal', n_bins=10,
                                                  strategy='uniform'),
                                 [0, 1, 2])])

In [ ]:
#Por posición con slice
make_column_transformer((discretizer,slice(0,3)))

ColumnTransformer(transformers=[('kbinsdiscretizer',
                                 KBinsDiscretizer(encode='ordinal', n_bins=10,
                                                  strategy='uniform'),
                                 slice(0, 3, None))])

In [ ]:
#Por indexación booleana
make_column_transformer((discretizer,x.columns.str.contains('3')))

ColumnTransformer(transformers=[('kbinsdiscretizer',
                                 KBinsDiscretizer(encode='ordinal', n_bins=10,
                                                  strategy='uniform'),
                                 array([False, False,  True, False, False, False, False, False, False]))])

In [ ]:
#Por un patrón en el nombre
from sklearn.compose import make_column_selector

make_column_transformer((discretizer,make_column_selector(pattern='3|4')))

ColumnTransformer(transformers=[('kbinsdiscretizer',
                                 KBinsDiscretizer(encode='ordinal', n_bins=10,
                                                  strategy='uniform'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000017C27E94D30>)])

In [ ]:
#Por un patrón de tipo de dato a incluir
from sklearn.compose import make_column_selector

make_column_transformer((discretizer,make_column_selector(dtype_include='number')))

ColumnTransformer(transformers=[('kbinsdiscretizer',
                                 KBinsDiscretizer(encode='ordinal', n_bins=10,
                                                  strategy='uniform'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000017C27E94730>)])

In [ ]:
#Por un patrón de tipo de dato a excluir
from sklearn.compose import make_column_selector

make_column_transformer((discretizer,make_column_selector(dtype_exclude='number')))

ColumnTransformer(transformers=[('kbinsdiscretizer',
                                 KBinsDiscretizer(encode='ordinal', n_bins=10,
                                                  strategy='uniform'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000017C27E94CA0>)])